# Imports

In [31]:
import xgboost as xgb
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# Train

In [32]:
train_set = pd.read_csv('train/train_encoded.csv')
train_set.head(10)

,len_location,len_keyword,len_text,len_text_original,diff_len_text,keyword_value,keyword_sum,keyword_min,keyword_max,keyword_mean,...,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v,text_value_1_gram,text_value_2_gram,text_value_3_gram,text_value_4_gram,target
0,0,0,43,69,26,3.267254,1.0,1.0,1.0,1.0,...,8.481658,0.000850,0.615265,0.154212,0.121727,0.226814,0.055999,0.0,0.0,1
1,0,0,36,38,2,3.267254,1.0,1.0,1.0,1.0,...,0.790154,-0.103024,0.308946,0.079015,0.055061,0.140340,0.059299,0.0,0.0,1
2,0,0,72,133,61,3.267254,1.0,1.0,1.0,1.0,...,29.265612,-0.044882,1.000000,0.139360,0.095317,0.323950,0.061498,0.0,0.0,1
3,0,0,50,65,15,3.267254,1.0,1.0,1.0,1.0,...,2.733583,-0.039545,0.317247,0.097628,0.082558,0.199979,0.056540,0.0,0.0,1
4,0,0,52,88,36,3.267254,1.0,1.0,1.0,1.0,...,13.153737,-0.041156,1.000000,0.125274,0.086994,0.258605,0.000000,0.0,0.0,1
5,0,0,84,110,26,3.267254,1.0,1.0,1.0,1.0,...,7.810325,-0.093557,0.478663,0.100132,0.080587,0.362519,0.086335,0.0,0.0,1
6,0,0,74,95,21,3.267254,1.0,1.0,1.0,1.0,...,9.341228,-0.047981,0.788929,0.141534,0.109943,0.320465,0.117669,0.0,0.0,1
7,0,0,21,59,38,3.267254,1.0,1.0,1.0,1.0,...,11.859967,-0.143466,1.000000,0.152051,0.112546,0.140984,0.029958,0.0,0.0,1
8,0,0,38,79,41,3.267254,1.0,1.0,1.0,1.0,...,15.609978,-0.033696,1.000000,0.200128,0.158737,0.180885,0.028352,0.0,0.0,1
9,0,0,24,52,28,3.267254,1.0,1.0,1.0,1.0,...,7.238668,0.004347,0.526257,0.160859,0.136866,0.119033,0.000000,0.0,0.0,1


### XGBoost

In [33]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Random Search

In [4]:
learning_rate = [0.1, 0.3, 0.01, 0.001]
max_depth = [20, 50, 70, 100]
#subsamples = [0.4, 0.5, 0.6]
colsample_bytree = [0.4, 0.5, 0.6]
n_estimators = [20, 50, 70, 100]
objective = ['reg:logistic']

random_search = {'learning_rate': learning_rate, 'max_depth': max_depth,\
                'colsample_bytree': colsample_bytree, 'n_estimators': n_estimators, 'objective': objective}

In [5]:
xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),\
                               param_distributions=random_search, n_iter=50, cv=7, random_state=111)

In [6]:
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                    

In [7]:
xgb_random.best_params_

{'objective': 'reg:logistic',
 'n_estimators': 100,
 'max_depth': 100,
 'learning_rate': 0.1,
 'colsample_bytree': 0.5}

In [8]:
preds = xgb_random.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.806671


In [34]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.6,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.815812


In [35]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.4,\
                           learning_rate = 0.001, max_depth = 50, n_estimators = 50)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.804431


In [36]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.811605


In [37]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.806599


Best score w/ test_size 0.1: 83,6

##### Best params de random forest

In [38]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', learning_rate = 0.1, max_depth = 70, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.816158


In [39]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.7,\
                           learning_rate = 0.1, max_depth = 37, n_estimators = 60)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.809284


In [40]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.798779


# Test

In [31]:
test_set = pd.read_csv('test/test_encoded.csv')

In [32]:
col_id = test_set['id']
test_set.drop('id', axis=1, inplace=True)

In [33]:
preds = xg_reg.predict(test_set)
preds

array([0, 0, 1, ..., 1, 1, 0])

In [34]:
test_set['target'] = preds
test_set['id'] = col_id
test_set[['id', 'target']]

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


### Resultado

In [35]:
test_set[['id', 'target']].to_csv('result.csv', header=True, index=False)